In [1]:
import csv
import pandas
import numpy
import random
from CWHashingFamily import CWHashingFamily

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

SHINGLES_LEN = 4
PRIME = 997
BINS = 900

def shingles(text, shingleSize):
    allShingles = []
    for i in xrange(10):
        randomEnd = random.randrange(shingleSize-1, len(text))
        allShingles.append(text[(randomEnd-shingleSize):randomEnd])
    return allShingles
    #return [text[i:i+shingleSize] for i in xrange(0,len(text)-(shingleSize+1))]


def minhash(shinglesList, seed):
    random.seed(seed)
    minhash = -1
    for shingle in shinglesList:
        cwFactory = CWHashingFamily(BINS, PRIME)
        numericHash = cwFactory.numericFunction(random.randrange(1,PRIME), random.randrange(0,PRIME))
        stringHash = cwFactory.stringFunction(random.randrange(0,PRIME),numericHash)
        shHash = stringHash.hash(shingle)
        if ((minhash == -1) or (shHash < minhash)):
            minhash = shHash
    return minhash

In [2]:
ID = 0
PREDICTION = 6
TEXT = 9

R = 4
B = 8

trainRDD = sc.textFile("train_sin_repeticiones.csv",minPartitions=None, use_unicode=False).mapPartitions(lambda x: csv.DictReader(x))
testRDD = sc.textFile("test.csv",minPartitions=None, use_unicode=False).mapPartitions(lambda x: csv.DictReader(x))

#testRDD = testRDD.filter(lambda line: line[ID] != "Id")
#trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")



In [11]:
trainRDD = sc.parallelize(trainRDD.top(10))
testRDD = sc.parallelize(testRDD.top(10))

trainRDD = trainRDD.map(lambda line: ((line["Id"],line["Prediction"]), shingles(line["Text"],SHINGLES_LEN)))
testRDD = testRDD.map(lambda line: (line["Id"], shingles(line["Text"],SHINGLES_LEN)))

trainRDD = trainRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] ))
testRDD = testRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] ))

print trainRDD.take(10)
print testRDD.take(10)

In [14]:
from LSH import LSH
from Record import Record
import time

lshTrain = LSH(R,B)

trainRDD = trainRDD.flatMap(lambda line: [lshTrain.add(Record(line[0][0], line[0][1], line[1]))])

#ESTO ANDA con la version anterior de LSH:

#trainRDD = trainRDD.take(300)

#start_time = time.time()

#for i in xrange(len(trainRDD)):
#    record = Record((trainRDD[i][0], trainRDD[i][1]) )
#    lshTrain.add(record)
    
#shinglesText = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."
#test = (492346,shingles(shinglesText,SHINGLES_LEN))
#test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B) ])
#testRecord = Record(test)

#print lshTrain.getAllSimilarRecords(testRecord)

#print("--- %s seconds ---" % (time.time() - start_time))


In [15]:
shinglesText = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."
test = (492346,shingles(shinglesText,SHINGLES_LEN))
test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B) ])
candidates = lshTrain.getAllSimilarRecords(Record(test[0], 0, test[1]))

In [16]:
def recordIsCandidate(candidates, record):
    for candidate in candidates:
        if record.isAt(candidate[0], candidate[1]):
            return True
    return False

trainRDD = trainRDD.filter(lambda record: recordIsCandidate(candidates, record)).flatMap(lambda record: [record.getPrediction()])

trainRDD = trainRDD.take(10)

for record in trainRDD:
    print str(record)